# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

# answer:

## 1

In [1]:
import pandas as pd
import operator
import json
from pandas.io.json import json_normalize
import numpy as np

In [2]:
df = pd.read_json('data/world_bank_projects.json')

In [3]:
country = df['countryname']

result_1={}

for count in country:
    result_1[count] = result_1.get(count,0)+1

result_1_order = sorted(result_1.items(),key=operator.itemgetter(1),reverse=True)
for i in range(10):
    print (result_1_order[i])

("People's Republic of China", 19)
('Republic of Indonesia', 19)
('Socialist Republic of Vietnam', 17)
('Republic of India', 16)
('Republic of Yemen', 13)
('Kingdom of Morocco', 12)
('Nepal', 12)
("People's Republic of Bangladesh", 12)
('Republic of Mozambique', 11)
('Africa', 11)


## 2

In [4]:
file_name = 'data/world_bank_projects.json'

#load json file.https://stackoverflow.com/questions/20199126/reading-json-from-a-file
with open(file_name) as json_data:
    js = json.load(json_data)
   

In [5]:
themes=json_normalize(js,'mjtheme_namecode')
themes

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [6]:
themes.replace('', np.nan, inplace=True)

In [7]:
themes.isnull().any()

code    False
name     True
dtype: bool

In [8]:
for idx, feature in enumerate(themes.columns):
    print (feature, ": ", themes.dtypes[idx])

code :  object
name :  object


In [9]:
result_theme = {}

for count in themes['code']:
    result_theme [count] = result_theme.get(count,0)+1
    
result_theme_order = sorted(result_theme.items(), key=operator.itemgetter(1), reverse=True)
for i in range(10):
    print (result_theme_order[i])

('11', 250)
('10', 216)
('8', 210)
('2', 199)
('6', 168)
('4', 146)
('7', 130)
('5', 77)
('9', 50)
('1', 38)


In [12]:
themes_name = themes.name.unique()
themes_name

array(['Human development', nan, 'Economic management',
       'Social protection and risk management', 'Trade and integration',
       'Public sector governance',
       'Environment and natural resources management',
       'Social dev/gender/inclusion',
       'Financial and private sector development', 'Rural development',
       'Urban development', 'Rule of law'], dtype=object)

In [13]:
#define the dataframe for the code-name lookup
df_code_name = pd.DataFrame()

for item in themes_name:
    if item == '':
        continue
        
    df_code_name = df_code_name.append(themes.loc[themes['name'] == item][:1])

In [14]:
df_code_name

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [15]:
result_2 = {}
for i in range(len(result_theme_order)):
    code,count = result_theme_order[i]
    theme_name = df_code_name.loc[df_code_name['code']==code]['name'].values[0]
    result_2[theme_name]=count

In [21]:
result_2_order = sorted(result_2.items(), key=operator.itemgetter(1), reverse=True)
for i in range(10):
    print (result_2_order[i])

('Environment and natural resources management', 250)
('Rural development', 216)
('Human development', 210)
('Public sector governance', 199)
('Social protection and risk management', 168)
('Financial and private sector development', 146)
('Social dev/gender/inclusion', 130)
('Trade and integration', 77)
('Urban development', 50)
('Economic management', 38)


## 3

In [17]:
#replace empty theme name with corresponding name
for index, row in themes.iterrows():
    
    if row.isnull().any():
        
        code = row['code']
        
        row['name'] = df_code_name.loc[df_code_name['code']==code]['name'].values[0]
        

In [18]:
themes.isnull().any()

code    False
name    False
dtype: bool

In [19]:
themes.head()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
